In [1]:
from pybaseball.lahman import *
import pandas as pd
import matplotlib.pylab as plt

class yearAvg:
    average = {}
    weights = {}

    def __init__(self, year):
        self.setAvgs(year)
        self.setWeight()

    def setAvgs(self, year):
        bat = batting()
        bat = bat.dropna()
        bat = bat.loc[(bat['yearID']==year) & (bat['AB']>0)]
        bat['BA'] = bat['H']/bat['AB']
        bat['ISO'] = (bat['2B']+2*bat['3B']+3*bat['HR'])/bat['AB']
        self.average['BA'] = bat['BA'].mean() # hits/AB - young
        self.average['ISO'] = bat['ISO'].mean() # young
        self.average['BB'] = bat['BB'].mean() # bases on balls (walks) - old
        self.average['SO'] = bat['SO'].mean() # strike outs - old
        self.average['SB'] = bat['SB'].mean() # stolen bases - young
        self.average['SF'] = bat['SF'].mean() # sacfriced flies - old

    def setWeight(self):
        for i in self.average:
            self.weights[i] = (4.20 / self.average[i]) # I want my scores to be avergeing around 4.20

    def getAvgs(self):
        return self.average
    def getWeight(self):
        return self.weights

In [2]:
class player:
    playerID = ''
    playerName = ''
    palyerBirthYear = 0
    yearsPlayed = []
    ChapoScore = 0
    yearISO = 0
    data = pd.DataFrame()


    def __init__(self, playerID):
        self.playerID = playerID
        self.setData(self.playerID)
        self.setInfo(self.playerID)

    def getStatcast(self, playerID):
        return dp.DataFrame()

    def setInfo(self, playerID):
        try:
            masterData = master()
            playerData = masterData.loc[masterData['playerID']==playerID].set_index('playerID')
            self.playerName = playerData['nameFirst'][playerID] + ' ' + playerData['nameLast'][playerID]
            self.palyerBirthYear = playerData['birthYear'][playerID]
            self.yearsPlayed = self.data['yearID'].tolist()
        except:
            print("Failed to find player")

    def getName(self):
        return self.playerName

    def setData(self, playerID):
        bat = batting()
        bat = bat.loc[bat['playerID']==playerID]
        bat['BA'] = bat['H']/bat['AB']
        bat['ISO'] = (bat['2B']+2*bat['3B']+3*bat['HR'])/bat['AB']
        self.data = bat[['playerID','yearID','BA','ISO','SO','BB','SB','SF']]

    def getData(self):
        return self.data

    def setChapoScore(self, year):
        # get the year averages for the used scores
        yearAverages = yearAvg(year)
        ya = yearAverages.getAvgs()
        # get all the data for one player in a specific year
        yearData = self.data.loc[self.data['yearID'] == year].set_index('playerID')
        age = yearData['yearID'][self.playerID] - self.palyerBirthYear
        age_weight = 0.00625*(float(age)-23)**3 + 1
        old_scores = yearData['SF'][self.playerID]/ya['SF']+yearData['BB'][self.playerID]/ya['BB']
        young_scores = yearData['ISO'][self.playerID]/ya['ISO'] + yearData['SB'][self.playerID]/ya['SB']
        self.yearISO = yearData['ISO'][self.playerID]
        try:
            self.ChapoScore = (young_scores*(age_weight) + old_scores*(2-age_weight))/4
        except:
            self.ChapoScore = 0

    def getYearsPlayed(self):
        return self.yearsPlayed
    def getChapoScore(self):
        return self.ChapoScore

    def getSummary(self):
        print("="*30)
        print(self.playerName, "Score Summary")
        print("="*30)
        for year in self.yearsPlayed:
            age = year - self.palyerBirthYear
            self.setChapoScore(year)
            print("age: ",age, "|",year,"score: ",self.getChapoScore(), "| ISO:",self.yearISO)

In [5]:
bat = batting()
colorado2016 = bat.loc[(bat['teamID']=='COL') & (bat['yearID']==2016)]['playerID'].tolist()
print(colorado2016)

['adamecr01', 'anderty01', 'arenano01', 'barnebr02', 'bergmch01', 'bettich01', 'blackch02', 'butleed01', 'carasma01', 'cardust01', 'castrmi01', 'chatwty01', 'dahlda01', 'delarjo01', 'descada01', 'estevca01', 'flandyo01', 'garnedu01', 'germego01', 'gonzaca01', 'grayjo02', 'gurkaja01', 'haleda02', 'hoffmje02', 'hundlni01', 'lemahdj01', 'loganbo02', 'lylesjo01', 'marquge01', 'mcgeeja01', 'milleju02', 'motteja01', 'murphto04', 'obergsc01', 'ottavad01', 'parrage01', 'pattejo03', 'paulsbe01', 'quallch01', 'raburry01', 'reynoma01', 'rusinch01', 'storytr01', 'tapiara01', 'valaipa01', 'wolteto01', 'ynoara01']


In [9]:
print("")
playerID = input("Enter a players playerID: ")
print("")
OldP = player(playerID)
OldP.getSummary()
print("")




Enter a players playerID: arenano01

Nolan Arenado Score Summary
age:  22.0 | 2013 score:  1.3915870295721455 | ISO: 0.13786008230452676
age:  23.0 | 2014 score:  2.2387366795631287 | ISO: 0.21296296296296297
age:  24.0 | 2015 score:  3.832926256195852 | ISO: 0.28733766233766234
age:  25.0 | 2016 score:  3.6219360545353196 | ISO: 0.2750809061488673



In [8]:
for i in colorado2016:
    OldP = player(i)
    OldP.getSummary()

Cristhian Adames Score Summary
age:  23.0 | 2014 score:  0.0 | ISO: 0.0
age:  24.0 | 2015 score:  0.19805333101447947 | ISO: 0.05660377358490566
age:  25.0 | 2016 score:  0.8174167678354329 | ISO: 0.08444444444444445
Tyler Anderson Score Summary
age:  27.0 | 2016 score:  0.33727893985945123 | ISO: 0.08571428571428572
Nolan Arenado Score Summary
age:  22.0 | 2013 score:  1.3915870295721455 | ISO: 0.13786008230452676
age:  23.0 | 2014 score:  2.2387366795631287 | ISO: 0.21296296296296297
age:  24.0 | 2015 score:  3.832926256195852 | ISO: 0.28733766233766234
age:  25.0 | 2016 score:  3.6219360545353196 | ISO: 0.2750809061488673
Brandon Barnes Score Summary
age:  26.0 | 2012 score:  0.3579951350925304 | ISO: 0.061224489795918366
age:  27.0 | 2013 score:  2.327761906147204 | ISO: 0.1053921568627451
age:  28.0 | 2014 score:  1.736000052355142 | ISO: 0.1678082191780822
age:  29.0 | 2015 score:  1.2914626130528917 | ISO: 0.09019607843137255
age:  30.0 | 2016 score:  1.0357339260479823 | ISO: 0

KeyboardInterrupt: 